<a href="https://colab.research.google.com/github/mopu3263/NLP-project_Tweets-Classificaiton/blob/main/Twitter_Kamala_Vs_Biden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Twitter comments detection: we will use tweepy, NLP for text preprocessing       and use scikit learn model to detect the tweets of a specific user**


---
# **Goals**
We will use Kamala and Biden Twitter accounts as example and will classify which tweet came from whom. The steps will be:


1.   Creat a methode using Tweepy to pull a list of tweets from twitter API
2.   Perform preprocessing with textacy library
3.   Use scikit supervised classification technique to identify the user

Some Twitter Rules
**bold text**
THERE ARE LIMITS TO THE AMOUNT OF TIMES YOU CAN HIT THE API PER 15 MINUTE WINDOW. BEWARE!

Here's a quick overview of what Twitter says are "the rules":
https://developer.twitter.com/en/docs/rate-limits

Tweepy installation & import tweepy, panda,nump etc

In [1]:
!pip install tweepy
import tweepy
import pandas as pd
import numpy as np
import re, datetime

Twitter is a great resource of text data for Natural Language Processing (NLP) project but required some works. The below website describes how to do step by step starting from application of developer account in Twitter

https://towardsdatascience.com/tweepy-for-beginners-24baf21f2c25

In [2]:
class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    twitter_keys = {
        'consumer_key':       'L7vVkmHZyAN1aEQ6KovMF72kr',
        'consumer_secret':     'Vq9W4dLPuuET08aR8D2ofcoZ4dhn1X8S37Ui1TqUlnrY7X6PLs',
        'access_token_key':    '935888914325184512-qGIL3l6bgrWrl4wPiy06CnGEkbDWlPR',
        'access_token_secret': '5hGsH2BI4W4BI53lHze54eUPEaAQu4J98hDJCtSELg8YZ'
    }
    
    
    def __init__(self, keys_dict=twitter_keys, api=api, result_limit = 20):
        
        self.twitter_keys = keys_dict
        
        auth = tweepy.OAuthHandler(keys_dict['consumer_key'], keys_dict['consumer_secret'])
        auth.set_access_token(keys_dict['access_token_key'], keys_dict['access_token_secret'])
        
        self.api = tweepy.API(auth)

        self.twitter_keys = keys_dict
        
        self.result_limit = result_limit
        

    def mine_user_tweets(self, user="HillaryClinton", #BECAUSE WHO ELSE!
                         mine_rewteets=False,
                         max_pages=5):

        data           =  []
        last_tweet_id  =  False
        page           =  1
        
        while page <= max_pages:
            if last_tweet_id:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                     count=self.result_limit,
                                                     max_id=last_tweet_id - 1,
                                                     tweet_mode = 'extended',
                                                     include_retweets=True
                                                    )        
            else:
                statuses   =   self.api.user_timeline(screen_name=user,
                                                        count=self.result_limit,
                                                        tweet_mode = 'extended',
                                                        include_retweets=True)
                
            for item in statuses:
                try:
                    mined = {
                        'tweet_id':        item.id,
                        'handle':          item.user.name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.full_text,
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item.created_at,
                    }
                    
                except:
                        mined = {
                        'tweet_id':        item.id,
                        'handle':          item.user.name,
                        'retweet_count':   item.retweet_count,
                        'text':            item.full_text,
                        'mined_at':        datetime.datetime.now(),
                        'created_at':      item.created_at,
                    }
                
                last_tweet_id = item.id
                data.append(mined)
                
            page += 1
            
        return data 

# **Instantiate** **the** **class** 

In [3]:
# Result limit == count parameter 200
miner=TweetMiner(result_limit = 200 )
biden = miner.mine_user_tweets(user="JoeBiden", max_pages=14)
kamala = miner.mine_user_tweets(user="KamalaHarris", max_pages=14)

In [4]:
for x in range(5):
    print (biden[x]['text'])
    print('---')

RT @POTUS: This afternoon, I’m visiting the @StateDept and delivering remarks on America’s role in the world. Tune in. https://t.co/zWG3UET…
---
RT @POTUS: Extending the moratorium on evictions and foreclosures to help countless Americans keep a roof over their head — that’s the Amer…
---
RT @POTUS: Let me be clear: The risk in this moment isn’t that we do too much — it’s that we don’t do enough. Congress must pass the Americ…
---
RT @POTUS: Officer Brian Sicknick was a hero who died in the line of duty defending our democracy and the citadel of liberty itself. To his…
---
RT @POTUS: No one should lose their home in the middle of this pandemic. My American Rescue Plan will extend the moratorium on evictions an…
---


In [5]:
for x in range(5):
    print (kamala[x]['text'])
    print('--')

RT @VP: We stand on the shoulders of those who came before. When Rosa Parks refused to give up her seat, she was lifting all of us up. Toda…
--
RT @POTUS: Extending the moratorium on evictions and foreclosures to help countless Americans keep a roof over their head — that’s the Amer…
--
RT @VP: This morning, I swore in Pete Buttigieg as Secretary of Transportation. @SecretaryPete is a true problem-solver. He will bring peop…
--
RT @VP: As Black History Month begins today, we remember and honor those who have come before, and we work to build a brighter future for a…
--
RT @VP: People are hurting right now. 

The American Rescue Plan gets economic relief to those who need it, and it gets people vaccinated.…
--


# **Converting tweet outputs to pandas DataFrame**

In [45]:
biden_df= pd.DataFrame(biden)
print(biden_df.shape)
biden_df.head(5)

(2800, 6)


,tweet_id,handle,retweet_count,text,mined_at,created_at
0,1357414638510616579,Joe Biden,3874,"RT @POTUS: This afternoon, I’m visiting the @S...",2021-02-05 17:24:32.651788,2021-02-04 19:44:04
1,1357376416380256258,Joe Biden,5825,RT @POTUS: Extending the moratorium on evictio...,2021-02-05 17:24:32.651802,2021-02-04 17:12:11
2,1357119042327359488,Joe Biden,20342,RT @POTUS: Let me be clear: The risk in this m...,2021-02-05 17:24:32.651804,2021-02-04 00:09:28
3,1357052766720557066,Joe Biden,12273,RT @POTUS: Officer Brian Sicknick was a hero w...,2021-02-05 17:24:32.651806,2021-02-03 19:46:07
4,1356962956647538688,Joe Biden,14317,RT @POTUS: No one should lose their home in th...,2021-02-05 17:24:32.651808,2021-02-03 13:49:14


In [7]:
kamala_df= pd.DataFrame(kamala)
print(kamala_df.shape)
kamala_df.head(5)

(2799, 6)


,tweet_id,handle,retweet_count,text,mined_at,created_at
0,1357443261049298946,Kamala Harris,11576,RT @VP: We stand on the shoulders of those who...,2021-02-05 17:24:39.542012,2021-02-04 21:37:48
1,1357399467209728000,Kamala Harris,5825,RT @POTUS: Extending the moratorium on evictio...,2021-02-05 17:24:39.542027,2021-02-04 18:43:46
2,1356989141561581570,Kamala Harris,9125,"RT @VP: This morning, I swore in Pete Buttigie...",2021-02-05 17:24:39.542029,2021-02-03 15:33:17
3,1356331078559158274,Kamala Harris,11743,"RT @VP: As Black History Month begins today, w...",2021-02-05 17:24:39.542031,2021-02-01 19:58:23
4,1356007024581107719,Kamala Harris,10863,RT @VP: People are hurting right now. \n\nThe ...,2021-02-05 17:24:39.542033,2021-01-31 22:30:42


In [8]:

tweets = pd.concat([biden_df, kamala_df], axis=0)
tweets.shape

(5599, 6)

# ***Pulling any interesting ngrams ***

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [10]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
summaries = "".join(biden_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)
Counter(ngrams_summaries).most_common(20)

[('donald trump', 344),
 ('covid 19', 184),
 ('president trump', 129),
 ('american people', 98),
 ('health care', 89),
 ('vote https', 73),
 ('white house', 73),
 ('head https', 68),
 ('make sure', 58),
 ('build better', 56),
 ('today https', 53),
 ('president ll', 50),
 ('climate change', 49),
 ('election day', 49),
 ('need president', 49),
 ('united states', 47),
 ('president https', 47),
 ('need help', 44),
 ('ve got', 44),
 ('soul nation', 39)]

In [33]:
vect = TfidfVectorizer(ngram_range=(2,5), stop_words='english')
summaries = "".join(kamala_df['text'])
ngrams_summaries = vect.build_analyzer()(summaries)
Counter(ngrams_summaries).most_common(20)

[('rt joebiden', 115),
 ('donald trump', 101),
 ('health care', 74),
 ('covid 19', 73),
 ('supreme court', 68),
 ('american people', 60),
 ('united states', 51),
 ('make sure', 50),
 ('trump administration', 48),
 ('care act', 46),
 ('white house', 45),
 ('wear mask', 40),
 ('affordable care', 40),
 ('affordable care act', 40),
 ('biden harris', 37),
 ('years ago', 37),
 ('elect joebiden', 35),
 ('breonna taylor', 35),
 ('black women', 34),
 ('election day', 34)]

### **Tweets preprocessing and developing a model**
To classify which tweets corresponding to whoe, the below steps are required:

1.   Vectorizing the input text
2.   Initializing the model
3.   Hyperparameter tuning with Grid Searching
4.   Model optimization
5.   Model performance evaluation



In [46]:
tweet_text = tweets['text'].values.tolist()
#creating target
y = tweets['handle'].map(lambda x: 1 if x == 'Joe Biden' else 0).values
print(y)


[1 1 1 ... 0 0 0]


In [47]:
#Vectorizing with TF-IDF Vectorizer and creating X matrix
tfv = TfidfVectorizer(ngram_range=(2,4), max_features=2000)
X = tfv.fit_transform(tweet_text).todense()
print (X.shape)

(5599, 2000)


# **We will use only SVC model for this project**

In [48]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV


In [49]:
params = {'kernel': ('linear', 'rbf'),'C': [1, 10]}
svr = svm.SVC()
#Grid searching to find optimal parameters for SVC Regression
clf = GridSearchCV(svr, params)
clf.fit(X, y)

GridSearchCV(cv=None, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [50]:
print (clf.best_params_)
print (clf.best_score_)

{'C': 10, 'kernel': 'rbf'}
0.7656715179369336


# **The below code might take several mintues so have a good cup of coffee or snackes you like**

In [51]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(svm.SVC(probability=True), X, y)

print (accuracies.mean())
print (1-y.mean())

0.7613883569513596
0.49991069833898916


# **Predicito of probability of Biden & Kamala Tweets**

# **Do not re-initialize the tfidf vectorizer otherwise the features will be overwritten and the transform will not match the number of feature with your trained model. We only need to transform since we already fit it before. **

In [54]:
estimator = svm.SVC(C=10, kernel='linear',probability=True)
estimator.fit(X,y)

SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [59]:
# Prepare the tweets as TfIdf vectors
source_test = [
   "Extending the moratorium on evictions and foreclosures to help countless Americans keep a roof over their head — that’s the American Rescue Plan.",
    "My abiding hope—my abiding prayer—is that we emerge from this ordeal with a new wisdom to cherish simple moments to imagine new possibilities and to open our hearts just a little more to one another."]

# I copied the these two comments from Joe Biden and Kamala Harris twitter accounts correspondingly

Xtest = tfv.transform(source_test).todense()

In [60]:
pd.DataFrame(estimator.predict_proba(Xtest),columns=["Proba_Biden", "Proba_Kamala"])

,Proba_Biden,Proba_Kamala
0,0.692913,0.307087
1,0.270381,0.729619


Based on the model the first tweet came from Biden is around 70% probable and the probability of 2nd comment coming from Kamala is 72%

# **Now I am going to pull out the tweets both from Joe Biden and Kamala Harris with the highest and lowest probability of being Bien or Kamala based on the model we developed above**

In [62]:
proba=pd.DataFrame(estimator.predict_proba(X),columns=["Proba_Biden", "Proba_Kamala"])

In [70]:
joined_x = pd.merge(tweets, proba, left_index=True, right_index=True)
joined_x

,tweet_id,handle,retweet_count,text,mined_at,created_at,Proba_Biden,Proba_Kamala
0,1357414638510616579,Joe Biden,3874,"RT @POTUS: This afternoon, I’m visiting the @S...",2021-02-05 17:24:32.651788,2021-02-04 19:44:04,0.042185,0.957815
0,1357443261049298946,Kamala Harris,11576,RT @VP: We stand on the shoulders of those who...,2021-02-05 17:24:39.542012,2021-02-04 21:37:48,0.042185,0.957815
1,1357376416380256258,Joe Biden,5825,RT @POTUS: Extending the moratorium on evictio...,2021-02-05 17:24:32.651802,2021-02-04 17:12:11,0.185163,0.814837
1,1357399467209728000,Kamala Harris,5825,RT @POTUS: Extending the moratorium on evictio...,2021-02-05 17:24:39.542027,2021-02-04 18:43:46,0.185163,0.814837
2,1357119042327359488,Joe Biden,20342,RT @POTUS: Let me be clear: The risk in this m...,2021-02-05 17:24:32.651804,2021-02-04 00:09:28,0.095864,0.904136
...,...,...,...,...,...,...,...,...
2797,1247296778095005696,Joe Biden,8383,"For months, I called on President Trump to ste...",2021-02-05 17:24:38.949881,2020-04-06 22:55:00,0.056593,0.943407
2797,1217820061441888261,Kamala Harris,7231,Last night’s Lev Parnas interview couldn’t hav...,2021-02-05 17:24:47.463318,2020-01-16 14:45:03,0.056593,0.943407
2798,1247283692231245824,Joe Biden,766,"Judy, from itsJudysLife, asked an important qu...",2021-02-05 17:24:38.949883,2020-04-06 22:03:00,0.119238,0.880762
2798,1217634074392743936,Kamala Harris,662,Low-income communities and communities of colo...,2021-02-05 17:24:47.463320,2020-01-16 02:26:00,0.119238,0.880762
